In [0]:
Databricks notebook source
MAGIC %md
MAGIC # Quickstart: Build and test simple Agent using Mosaic AI Agent Framework
MAGIC - This quickstart notebook demonstrates how to build and test a generative AI agent using Mosaic AI Agent Framework on Databricks
MAGIC - Automatically log traces from LLM calls for ease of debugging using Mlflow
MAGIC - The agent uses an LLM served on Databricks Foundation Model API
MAGIC - The agent has access to a single tool, the built-in Python code interpreter tool on Databricks Unity Catalog. It can use this tool to run LLM-generated code in order to respond to user questions.
MAGIC
MAGIC We will use databricks_openai SDK to query the LLM endpoint.
MAGIC

In [0]:


# COMMAND ----------

# MAGIC %pip list #check what is already installed
%pip list



Package                            Version
---------------------------------- -------------------------------
annotated-types                    0.7.0
anyio                              4.6.2
argon2-cffi                        21.3.0
argon2-cffi-bindings               21.2.0
arrow                              1.3.0
asttokens                          2.0.5
astunparse                         1.6.3
async-lru                          2.0.4
attrs                              24.3.0
azure-common                       1.1.28
azure-core                         1.34.0
azure-identity                     1.20.0
azure-mgmt-core                    1.5.0
azure-mgmt-web                     8.0.0
azure-storage-blob                 12.23.0
azure-storage-file-datalake        12.17.0
babel                              2.16.0
beautifulsoup4                     4.12.3
black                              24.10.0
bleach                             6.2.0
blinker                            1.7.0
boto3          

In [0]:
# COMMAND ----------

# MAGIC %pip install -U -qqqq mlflow databricks-openai databricks-agents
%pip install -U -qqqq mlflow databricks-openai databricks-agents


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
# COMMAND ----------

dbutils.library.restartPython()



In [0]:
# COMMAND ----------

# MAGIC %pip show databricks-agents
%pip show databricks-agents


Name: databricks-agents
Version: 1.8.2
Summary: Mosaic AI Agent Framework SDK
Home-page: 
Author: Databricks
Author-email: feedback@databricks.com
License: Databricks Proprietary License
Location: /local_disk0/.ephemeral_nfs/envs/pythonEnv-05ad9df2-c959-4c23-b5fc-970ba416d233/lib/python3.12/site-packages
Requires: boto3, botocore, databricks-connect, databricks-sdk, dataclasses-json, jinja2, litellm, mlflow-skinny, pydantic, tenacity, tiktoken, tqdm, urllib3, whenever
Required-by: 


In [0]:
# COMMAND ----------

# DBTITLE 1,Suppress warnings globally
import warnings
warnings.filterwarnings('ignore')

In [0]:


# COMMAND ----------

# MAGIC %md
# MAGIC ## Pick the first LLM API available in your Databricks workspace

# COMMAND ----------

# The snippet below tries to pick the first LLM API available in your Databricks workspace
# from a set of candidates. You can override and simplify it
# to just specify LLM_ENDPOINT_NAME.
LLM_ENDPOINT_NAME = None

from databricks.sdk import WorkspaceClient
def is_endpoint_available(endpoint_name):
  try:
    client = WorkspaceClient().serving_endpoints.get_open_ai_client()
    client.chat.completions.create(model=endpoint_name, messages=[{"role": "user", "content": "What is AI?"}])
    return True
  except Exception:
    return False
  
client = WorkspaceClient()
for candidate_endpoint_name in ["databricks-claude-3-7-sonnet", "databricks-meta-llama-3-3-70b-instruct"]:
    if is_endpoint_available(candidate_endpoint_name):
      LLM_ENDPOINT_NAME = candidate_endpoint_name
assert LLM_ENDPOINT_NAME is not None, "Please specify LLM_ENDPOINT_NAME"



In [0]:
# COMMAND ----------

LLM_ENDPOINT_NAME



'databricks-meta-llama-3-3-70b-instruct'

In [0]:
# COMMAND ----------

# MAGIC %md
# MAGIC ## Lets first do a LLM call

# COMMAND ----------

import json
import mlflow
from databricks.sdk import WorkspaceClient

# Automatically log traces from LLM calls for ease of debugging
mlflow.openai.autolog()

# Get an OpenAI client configured to talk to Databricks model serving endpoints
# We'll use this to query an LLM in our agent
openai_client = WorkspaceClient().serving_endpoints.get_open_ai_client()

prompt = "What is the capital of India ?"

openai_client.chat.completions.create(
        model=LLM_ENDPOINT_NAME,
        messages=[{"role": "user", "content": prompt}],
    )



ChatCompletion(id='chatcmpl_9ba835bb-8f1a-4f82-9416-4fd36262219a', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The capital of India is New Delhi.', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1764650292, model='meta-llama-3.3-70b-instruct-121024', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=9, prompt_tokens=17, total_tokens=26, completion_tokens_details=None, prompt_tokens_details=None))

Trace(trace_id=tr-558b9b763f1b0afe01583399640645d4)

In [0]:
# COMMAND ----------

response = openai_client.chat.completions.create(
        model=LLM_ENDPOINT_NAME,
        messages=[{"role": "user", "content": prompt}],
    )



Trace(trace_id=tr-c5d5179a920980652cb260b209144f14)

In [0]:
# COMMAND ----------

response

ChatCompletion(id='chatcmpl_b6a5434c-107a-4147-b1fd-a2ebee39403c', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The capital of India is New Delhi.', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1764650424, model='meta-llama-3.3-70b-instruct-121024', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=9, prompt_tokens=17, total_tokens=26, completion_tokens_details=None, prompt_tokens_details=None))

In [0]:
# COMMAND ----------

response.id

'chatcmpl_b6a5434c-107a-4147-b1fd-a2ebee39403c'

In [0]:
# COMMAND ----------

response.choices[0].message.content

'The capital of India is New Delhi.'

In [0]:
# COMMAND ----------

# MAGIC %md
# MAGIC ### Now lets wrap in function

In [0]:


# COMMAND ----------

def run_llm(prompt):
    """
    Send a user prompt to the LLM, and return a list of LLM response messages
    The LLM is allowed to call the code interpreter tool if needed, to respond to the user
    """
    # Initialize an empty list to store response messages
    result_msgs = []
    # Send the user prompt to the LLM endpoint and get the response
    response = openai_client.chat.completions.create(
        model=LLM_ENDPOINT_NAME,
        messages=[{"role": "user", "content": prompt}],
    )
    # Extract the first message from the response
    msg = response.choices[0].message
    # Convert the message to a dictionary and add it to the result list
    result_msgs.append(msg.to_dict())

    # Return the list of response messages
    return result_msgs



In [0]:
# COMMAND ----------

resp = run_llm("What is the capital of India ?")
for message in resp:
    print(f'{message["role"]}: {message["content"]}')




assistant: The capital of India is New Delhi.


Trace(trace_id=tr-46977a2017e92aab8fe2d4d6986d14f3)

In [0]:
# COMMAND ----------

# MAGIC %md
# MAGIC ## Let's add tool and create a Agent

# COMMAND ----------

from databricks_openai import UCFunctionToolkit, DatabricksFunctionClient
from databricks.sdk import WorkspaceClient

# Automatically log traces from LLM calls for ease of debugging
mlflow.openai.autolog()

# Get an OpenAI client configured to talk to Databricks model serving endpoints
# We'll use this to query an LLM in our agent
openai_client = WorkspaceClient().serving_endpoints.get_open_ai_client()

# Load Databricks built-in tools (a stateless Python code interpreter tool)
client = DatabricksFunctionClient()
builtin_tools = UCFunctionToolkit(
    function_names=["system.ai.python_exec"], client=client
).tools
#for tool in builtin_tools:
#   del tool["function"]["strict"]


def call_tool(tool_name, parameters):
    if tool_name == "system__ai__python_exec":
        return DatabricksFunctionClient().execute_function(
            "system.ai.python_exec", parameters=parameters
        )
    raise ValueError(f"Unknown tool: {tool_name}")


def run_agent(prompt):
    """
    Send a user prompt to the LLM, and return a list of LLM response messages
    The LLM is allowed to call the code interpreter tool if needed, to respond to the user
    """
    result_msgs = []
    response = openai_client.chat.completions.create(
        model=LLM_ENDPOINT_NAME,
        messages=[{"role": "user", "content": prompt}],
        tools=builtin_tools,
    )
    msg = response.choices[0].message
    result_msgs.append(msg.to_dict())

    # If the model executed a tool, call it
    if msg.tool_calls:
        call = msg.tool_calls[0]
        tool_result = call_tool(call.function.name, json.loads(call.function.arguments))
        result_msgs.append(
            {
                "role": "tool",
                "content": tool_result.value,
                "name": call.function.name,
                "tool_call_id": call.id,
            }
        )
    return result_msgs



In [0]:
# COMMAND ----------

answer = run_agent("What is the best place to visit in India ?")

#answer = run_agent("What is the square root of 429?")
for message in answer:
    print(f'{message["role"]}: {message["content"]}')



assistant: India is a vast and diverse country with a wide range of exciting destinations to explore. The best place to visit in India depends on your interests, preferences, and the time of year you plan to visit. However, some of the most popular and iconic destinations in India include:

1. The Taj Mahal in Agra: A stunning white marble mausoleum and one of the Seven Wonders of the World.
2. The Golden Temple in Amritsar: A beautiful and historic gurudwara (Sikh temple) known for its stunning architecture and spiritual significance.
3. The beaches of Goa: A tropical paradise with beautiful beaches, vibrant nightlife, and a laid-back atmosphere.
4. The backwaters of Kerala: A network of tranquil waterways and lakes that offer a unique and relaxing experience.
5. The cities of Delhi and Mumbai: Bustling metropolises with a rich history, cultural attractions, and a vibrant food scene.
6. The Himalayan mountains: A range of majestic mountains that offer trekking, hiking, and skiing oppo

Trace(trace_id=tr-2229483befeccfdb56512b6c92caca63)

In [0]:
# COMMAND ----------

import json
answer = run_agent("What is the square root of 9?")
for message in answer:
    print(f'{message["role"]}: {message["content"]}')

# COMMAND ----------

# MAGIC %md
# MAGIC ## Next steps you could explore
# MAGIC - log the agent
# MAGIC - deploy
# MAGIC - Use that in the AI playground or app

assistant: None
tool: 3.0



Trace(trace_id=tr-8403509f62156af6da612de837875159)

In [0]:
Next Steps will be:
    - Log the agent
    - Deploy
    - Use that in the AI playground or app